# Legal-RAG: Colab Demo

1. 克隆 Legal-RAG 仓库
2. 安装依赖 
3. 设置LLM， 如 Qwen2-1.5B-Instruct
4. 构建索引
5. 通过 `RagPipeline` 对真实法律问题进行问答

---
在 Colab 中：
- `Runtime` → `Change runtime type` → `Hardware accelerator` 选择 `GPU`
- 如果使用 Hugging Face Hub 下载 Qwen 模型，可以在环境变量中配置 `HF_TOKEN`

In [ ]:
!nvidia-smi || echo "No GPU detected."

## 1. 克隆仓库 & 安装依赖


In [ ]:
import os
from pathlib import Path

ROOT = Path("/content").resolve()
os.chdir(ROOT)
print("ROOT:", ROOT)

REPO_URL = "https://github.com/Fan-Luo/Legal-RAG.git"  # 仓库地址
REPO_DIR = ROOT / "Legal-RAG"
os.environ["REPO_DIR"] = str(REPO_DIR)

if not REPO_DIR.exists():
    !git clone "$REPO_URL" "$REPO_DIR"
else:
    %cd "$REPO_DIR"
    !git pull

%cd "$REPO_DIR"
print("Current dir:", Path.cwd())

In [ ]:
%%bash
set -e
cd "$REPO_DIR" || exit 1

pip install -q -r requirements.txt
# 保险起见补充：transformers / accelerate / bitsandbytes
pip install -q transformers accelerate bitsandbytes

echo "Dependencies installed."

## 2. 准备数据 & 构建索引（如已构建可跳过）

需要：
- `data/raw/minfadian_hetongbian.txt`（可以手动上传后复制到该路径）
- 然后运行预处理脚本和建索引脚本。

In [ ]:
from pathlib import Path
import os

%cd "$REPO_DIR"

RAW_DIR = Path("data/raw")
RAW_DIR.mkdir(parents=True, exist_ok=True)
print("Raw dir:", RAW_DIR.resolve())
print("Raw files:", list(RAW_DIR.iterdir()))

# 如有需要，运行预处理与建索引脚本
try:
    !python -m scripts.preprocess_law
except SystemExit:
    print("Preprocessing completed.")

try:
    !python -m scripts.build_index
except SystemExit:
    print("Index building completed.")

print("Processed:")
!ls -R data/processed || echo "no processed dir"
print("Index:")
!ls -R data/index || echo "no index dir"

## 3. 配置 LLM（在 Notebook 里切换模型）

这里通过 `AppConfig.load()` 加载默认配置，然后在 Notebook 中修改：

- `cfg.llm.provider = "qwen-local"`
- `cfg.llm.model = "Qwen/Qwen2-1.5B-Instruct"`

也可以改成 OpenAI 的模型：

```python
cfg.llm.provider = "openai"
cfg.llm.model = "gpt-4.1-mini"  # 需要在环境变量设置 OPENAI_API_KEY
```

In [ ]:
from legalrag.config import AppConfig
from legalrag.pipeline.rag_pipeline import RagPipeline
# 使用 AppConfig.load()，自动补全各类路径并创建目录
cfg = AppConfig.load()
# 在 Notebook 里配置要使用的模型
cfg.llm.provider = "qwen-local"  # 或 "openai"
cfg.llm.model = "Qwen/Qwen2-1.5B-Instruct"  # 可改成其他 Qwen 模型或本地路径
print("LLM provider:", cfg.llm.provider)
print("LLM model   :", cfg.llm.model)
pipeline = RagPipeline(cfg)
def ask(question: str, top_k: int = 8):
    print("问题：", question)
    print("=" * 80)
    ans = pipeline.answer(question, top_k=top_k)
    print("【回答】\n")
    print(ans.answer)
    print("\n【命中条文】\n")
    for h in ans.hits:
        c = h.chunk
        print(f"[{h.rank}] 第{c.article_no}条 ({c.chapter} / {c.section}) score={h.score:.4f}")
        preview = (c.text or "")[:80].replace("\n", " ")
        print("   ", preview, "...")
        print()q1 = "合同约定的违约金为合同金额的 40%，是否合理？"
ask(q1, top_k=8)